# Amazon SageMaker - Bring Your Own Model (Deploy)
## MXNet + Gluon 編

MXNet の Gluon API では、Deep Learning の実装を簡単にするだけでなく、すでに学習済みのモデルも提供しています。学習済みのモデルを使うことによって、ユーザは学習を行うことなく、機械学習を利用することができます。Gluon では、コンピュータービジョンのための [GluonCV](https://gluon-cv.mxnet.io/)、自然言語処理のための [GluonNLP](https://gluon-nlp.mxnet.io/)、時系列データ予測のための [GluonTS](https://gluon-ts.mxnet.io/) を提供しています。

このノートブックでは、GluonCV を利用して学習済みのモデルを取得し、そのモデルをデプロイするための実装を行います。


## 1.学習済みのモデルのダウンロード
### GluonCVのダウンロード
デプロイする学習済みのモデルを取得するために、GluonCV をノートブックインスタンスにダウンロードします。`!`を冒頭に入れてインストールコマンドを実行します。

In [ ]:
!pip install gluoncv

### 学習済みモデルのダウンロード・保存

GluonCV を import して、`model_zoo.get_model`を行うことでモデルを読み込むことができます。`pretrained=True` と指定することで、学習済みモデルを利用することができます。利用可能なモデルについては、https://gluon-cv.mxnet.io/api/model_zoo.html　から参照できます。

In [ ]:
from gluoncv import model_zoo, data, utils
detector = model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)

モデルはまだメモリに読み込まれた状態ですので、デプロイするためには、このモデルをファイルに保存する必要があります。まず読み込んだモデル `detector` に対して `hybridize()`を実行します。MXNet では、デフォルトではコーディングとデバッグが容易な imperative という形式をとりますが、実行速度やモデルのサイズという観点で最適というわけではありません。そこで、保存のまえに`hybridize()`を実行して、モデルを最適な形に変換します。詳細は[こちら](https://mxnet.incubator.apache.org/versions/master/tutorials/gluon/hybrid.html)をご覧ください。

`hybridize()`自体は、効率的なモデルに変換しただけで、そのモデルのリソースはまだ未定なままです。データを入力して初めてリソースが決まり、そのモデルを保存することができます。ここでは、すべて1で次元が(1,3, 320, 320)のテンソルを入力します。

あとは、`export`関数によってモデルを保存します。ここでは保存先・ファイル名は以下の通りとします。
- model/detector-symbol.json
- model/detector-0000.params


In [ ]:
import mxnet as mx
import os
detector.hybridize()
detector(mx.nd.ones((1,3, 320, 320)))


model_dir = "model"
os.makedirs(model_dir, exist_ok=True)

detector_model = "detector"
detector.export("{}/{}".format(model_dir, detector_model))

### モデルのアップロード

保存が終わったら、SageMaker が読めるように、これを`tar.gz`の形式に圧縮して Amazon S3 にアップロードします。

In [ ]:
archive = 'model.tar.gz'
!tar cvzf $archive $model_dir

import sagemaker
sagemaker_session = sagemaker.Session()
model_uri = sagemaker_session.upload_data(path=archive, key_prefix='detector/model')
print('model archive is uploaded to {}'.format(model_uri))

## 2.デプロイ用のコード作成


### 必要な実装
モデルが作成できたら、次にデプロイ用のコードを書きます。MXNetの場合、デプロイに必要な関数は以下の通りです ([詳細](https://sagemaker.readthedocs.io/en/stable/using_mxnet.html#the-sagemaker-mxnet-model-server))

- model_fn (required)  
デプロイしたモデルを読み込む関数

- input_fn/predict_fn/output_fn (option)  
推論のリクエストを受け付けたとき、モデルで予測する前のデータ処理を input_fn で行い、predict_fn で予測し、ouput_fn で後処理を行います。実行の流れは以下のようになります。

```python
# Deserialize the Invoke request body into an object we can perform prediction on
input_object = input_fn(request_body, request_content_type)

# Perform prediction on the deserialized object, with the loaded model
prediction = predict_fn(input_object, model)

# Serialize the prediction result into the desired response content type
ouput = output_fn(prediction, response_content_type)
```

- transform_fn  (option)  
input_fn/predict_fn/output_fn を1つの関数で書く場合は transform_fn を使用します。

input_fn/predict_fn/output_fn と transform_fn は併用できません。これらの実装が required でない理由として、コンテナ側に標準の実装が行われているからです。例えば、json データの parse は標準的に実装されています。

ここでは、`model_fn` と `transform_fn` を実装してみましょう。

### スクリプトの準備

ここでは `detector.py` というスクリプトを用意します。まずは以下のコマンドで空のファイルを作成し、以降の作業で実装を行っていきます。

In [ ]:
!touch detector.py

### model_fn の実装

`model_fn`は、先ほどS3にアップロードしたモデルを読み込む実装となります。S3にモデルをアップロードしていれば、`model_dir` の場所に、ファイルがダウンロード・展開されます。以下のように`gluon.nn.SymbolBlock.imports`を呼び出して実行するコードを`detector.py`に実装しましょう。ファイル名が、保存したファイル名と同じになるようにします。ここでは後で必要なライブラリも import しておきます。

```python
from __future__ import print_function

import logging
import os
import numpy as np

import mxnet as mx
from mxnet import gluon, autograd
import json

logging.basicConfig(level=logging.DEBUG)

def model_fn(model_dir):
    logging.info(os.listdir(model_dir+"/model"))
    net = gluon.nn.SymbolBlock.imports(model_dir+ '/model/detector-symbol.json', 
                                       ['data'], 
                                       model_dir+ '/model/detector-0000.params')
    return net

```

### transform_fn の実装

次に `transform_fn` を`detector.py`に実装します。`transform_fn`では、先ほど読み込んだモデルを `net` として受け取ることができます。`data`は推論のリクエストのデータです。


```python
def transform_fn(net, data, input_content_type, output_content_type):
    data = json.loads(data)
    nda = mx.nd.array(data)
    class_IDs, scores, bounding_boxs = net(nda)
    
    output_list = []
    for i in range(class_IDs.shape[0]):
        exist_IDs = np.where(class_IDs[i,:,0].asnumpy() >= 0)
        output = {
            "class_ids": class_IDs[i,exist_IDs].asnumpy().tolist(),
            "scores": scores[i,exist_IDs].asnumpy().tolist(),
            "bbox": bounding_boxs[i,exist_IDs].asnumpy().tolist()
        }
        output_list.append(output)

    response_body = json.dumps(output_list)
    return response_body, output_content_type
```


まずは`data`に対して前処理を行います。ここでは json 形式でデータを受け取る前提で実装します。そこで `json.loads(data)`で json データを読み込み、Gluonが利用できるようにMXNet の ndrrayの形式に変換します。

推論自体は `net(nda)` だけで完了です。推論の結果は、画像に存在するクラスのID, そのスコア、バウンディングボックス (位置）です。

これらの情報をそのまま json 形式にして `return` つまり、クライアントにレスポンスとして返してもかまいませんが、少し後処理を加えましょう。 この学習済みのモデルでは、最大100個のオブジェクトを検出することができ、出力として100個のデータを返します。しかし、実際にはもっと少数のオブジェクトが検出されるでしょう。検出されたオブジェクトは正のオブジェクトIDで出力されるので、以下の関数で検出されたオブジェクトのみに絞ります。

```
exist_IDs = np.where(class_IDs[i,:,0].asnumpy() >= 0)
```

これらのIDに絞って json 形式とし、return してクライアントに返します。MXNet の ndarray は json に変換できないので、`.asnumpy().tolist()`でリストにします。

## 3.デプロイ

デプロイは、S3にアップロードしたモデルを指定して、`deploy`関数を実行します。MXNetの場合は、`MXNetModel`でモデルの指定を行います。
まずコードにエラーがないか確認するために、`local`モードでデプロイします。`local`モードはこのノートブックインスタンスでデプロイすることを意味します。インスタンスの立ち上げが不要なので、待ち時間が発生せず、効率よくデバッグできます。

問題がないことを確認できたら、`instance_type` にSagemakerのインスタンスに設定して、APIをたてることができます。

In [ ]:
from sagemaker.mxnet.model import MXNetModel
from sagemaker import get_execution_role


model = MXNetModel(model_data = model_uri,
                       role = get_execution_role(),
                       entry_point = "detector.py",
                        framework_version='1.4',
                       py_version='py3')
predictor = model.deploy(instance_type="local", initial_instance_count=1)



試しに1枚の画像を送ってみます。

In [ ]:
%matplotlib inline

import numpy as np
from PIL import Image, ImageFilter
from matplotlib import pyplot as plt

im = Image.open('images/dog1.jpg')
im = im.resize((320, 320), Image.LANCZOS)
data = np.array(im)/255
data = np.transpose(data, [2,0,1])
data = np.expand_dims(data, axis=0)
result = predictor.predict(data.tolist())

img =  np.transpose(np.array(im), [2,0,1])
ax = utils.viz.plot_bbox(np.array(im),
                         labels = np.array(result[0]["class_ids"]).reshape(-1),
                         scores = np.array(result[0]["scores"]).reshape(-1),
                         bboxes = np.array(result[0]["bbox"][0]),
                         class_names=detector.classes)
plt.show()

不要になったらエンドポイントを削除しましょう。

In [ ]:
predictor.delete_endpoint()

## 4.バッチ推論を行う

### ファイルのアップロード
バッチ変換ジョブを利用することで、Amazon S3にあるファイルを一括で推論することができます。ここでは `images` にある jpg ファイルに対して推論を行います。まず、これらのファイルを S3 にアップロードしましょう。

In [ ]:
image_uri = sagemaker_session.upload_data(path="./images", key_prefix='detector/images')

### デプロイのスクリプトの変更

バッチ変換ジョブでは、jpg ファイルを直接処理しますので、jpg ファイルの読み込みと前処理を実装する必要があります。ファイルタイプによって処理を分けたい場合は、`input_content_type`ごとに処理を記述します。`detector.py`のtransform_fnを以下の実装に変更しましょう。


```python
def transform_fn(net, data, input_content_type, output_content_type):

    from PIL import Image
    from six import BytesIO

    if input_content_type == "application/json":
        data = json.loads(data)
    elif input_content_type == 'image/jpeg':
        im = Image.open(BytesIO(data))
        im = im.resize((320, 320), Image.LANCZOS)
        data = np.array(im)/255
        data = np.transpose(data, [2,0,1])
        data = np.expand_dims(data, axis=0)

    nda = mx.nd.array(data)
    class_IDs, scores, bounding_boxs = net(nda)
    
    output_list = []
    for i in range(class_IDs.shape[0]):
        exist_IDs = np.where(class_IDs[i,:,0].asnumpy() >= 0)
        output = {
            "class_ids": class_IDs[i,exist_IDs].asnumpy().tolist(),
            "scores": scores[i,exist_IDs].asnumpy().tolist(),
            "bbox": bounding_boxs[i,exist_IDs].asnumpy().tolist()
        }
        output_list.append(output)
            
    response_body = json.dumps(output_list)
    return response_body, output_content_type
```

## バッチ変換ジョブの実行

デプロイの場合と同様にモデルを読み込んでからバッチ変換ジョブを実行します。バッチ変換ジョブは Transformer クラスを定義して実行しますが、その前に create_model でSageMaker モデルを作成しておく必要があります。


In [ ]:
from sagemaker.mxnet.estimator import MXNet
from sagemaker.transformer import Transformer
from sagemaker.utils import sagemaker_timestamp

output_path = 's3://' +sagemaker_session.default_bucket() + '/batch_transform/output'

model = MXNetModel(model_data = model_uri,
                   name ="gluon-batch-"+sagemaker_timestamp(),
                   role = get_execution_role(),
                   sagemaker_session = sagemaker_session,
                   entry_point = "detector.py",
                   framework_version='1.4',
                   py_version='py3')

container_defs = model.prepare_container_def(instance_type='ml.m4.xlarge')
sagemaker_session.create_model(model.name,
                              role =  get_execution_role(),
                               container_defs = container_defs
                              )
transformer =Transformer(model_name=model.name,
                                instance_count=1,
                                output_path=output_path,
                                instance_type='ml.m4.xlarge')
transformer.transform(image_uri, content_type='image/jpeg')

### 実行結果のダウンロード

S3 から推論結果をダウンロードしてみてみます。

In [ ]:
!aws s3 cp $output_path ./output --recursive